<p style="font-size: 24px; font-weight: bold;">Hello there!</p>

<p style="font-size: 16px;">This notebook introduces a super simple way to create a submission file for the competition of <b>"Parkinson's Freezing of Gait Prediction"</b>.</p>

<p style="font-size: 16px;">In this notebook, you will create features by combining 3-dimensional accelerometer data values with metadata about subjects in order to detect FoG (Freezing of Gait) events.</p>

<p style="font-size: 16px;">To predict FoG events (<code>'StartHesitation'</code>, <code>'Turn'</code>, <code>'Walking'</code>) at a given time point <code><b>t</b></code>, you will use the accelerometer data values and subject information at the same time point <code><b>t</b></code>.</p>

<p style="font-size: 16px;">However, since this procedure does not handle temporal information well, it must be necessary to conduct innovative feature engineering to achieve better performance.</p>

<p style="font-size: 16px;">The purpose of publishing this notebook is to demonstrate the rough procedure up to submitting results for the competition using simple code as much as possible.</p>

<p style="font-size: 16px;">I hope that the release of this notebook will contribute even a little to the excitement of the competition.</p>

<p style="font-size: 16px;">Let's enjoy Kaggle together!</p>

<h1>Import Modules</h1>

In [1]:
import os
import tqdm
import glob
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

In [2]:
# parent directory
pdir = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction'

# load meta data

In [3]:
df_tdcs_meta = pd.read_csv(os.path.join(pdir, 'tdcsfog_metadata.csv'))
df_tdcs_meta.head()

,Id,Subject,Visit,Test,Medication
0,003f117e14,4dc2f8,3,2,on
1,009ee11563,f62eec,4,2,on
2,011322847a,231c3b,2,2,on
3,01d0fe7266,231c3b,2,1,off
4,024418ba39,fa8764,19,3,on


In [4]:
df_defog_meta = pd.read_csv(os.path.join(pdir, 'defog_metadata.csv'))
df_defog_meta.head()

,Id,Subject,Visit,Medication
0,02ab235146,e1f62e,2,on
1,02ea782681,ae2d35,2,on
2,06414383cf,8c1f5e,2,off
3,092b4c1819,2874c5,1,off
4,0a900ed8a2,0e3d49,2,on


In [5]:
df_subjects = pd.read_csv(os.path.join(pdir, 'subjects.csv'))
df_subjects.head()

,Subject,Visit,Age,Sex,YearsSinceDx,UPDRSIII_On,UPDRSIII_Off,NFOGQ
0,00f674,2.0,63,M,27.0,43.0,49.0,24
1,00f674,1.0,63,M,27.0,31.0,30.0,26
2,02bc69,NaN,69,M,4.0,21.0,NaN,22
3,040587,2.0,75,M,26.0,52.0,69.0,21
4,040587,1.0,75,M,26.0,47.0,75.0,24


# Load tdcsfog data

In [6]:
# list of all tdcsfog csv file path
tdcs_file_path = glob.glob(os.path.join(pdir, 'train', 'tdcsfog', '*.csv'), recursive=True)

# In this notebook, we limit the number of files to be read in order to reduce the time required for model training.
# tdcs_file_path = tdcs_file_path[::1]

print(f'the number of files to be read: {len(tdcs_file_path)}')

the number of files to be read: 833


In [7]:
# Initialize a DataFrame to combine data from multiple CSV files.
df_tdcs = pd.DataFrame()

# load tdcsfog time series in combination with metadata.
for fp in tqdm.tqdm(tdcs_file_path):    
    
    # load data into a variable 'tmp'.
    tmp = pd.read_csv(fp)
    
    # get file Id from csv file name.
    file_id = os.path.basename(fp).replace(".csv", "")
    
    # get subject Id.
    subject = df_tdcs_meta.loc[df_tdcs_meta['Id'] == file_id, 'Subject'].iloc[0]
    
    # add metadata.
    tmp['Medication'] = df_tdcs_meta.loc[df_tdcs_meta['Id'] == file_id, 'Medication'].iloc[0]
    tmp['Age'] = df_subjects.loc[df_subjects['Subject'] == subject, 'Age'].iloc[0]
    tmp['Sex'] = df_subjects.loc[df_subjects['Subject'] == subject, 'Sex'].iloc[0]
    tmp['YearsSinceDx'] = df_subjects.loc[df_subjects['Subject'] == subject, 'YearsSinceDx'].iloc[0]
    tmp['NFOGQ'] =df_subjects.loc[df_subjects['Subject'] == subject, 'NFOGQ'].iloc[0]
    
    # concat the data
    df_tdcs = pd.concat([df_tdcs, tmp]).reset_index(drop=True)

100%|██████████| 833/833 [18:30<00:00,  1.33s/it]


In [ ]:
# check the contents of the df_tdcs
df_tdcs.head()

# Load defog data

In [ ]:
# list of all tdcsfog csv file path
defog_file_path = glob.glob(os.path.join(pdir, 'train', 'defog', '*.csv'), recursive=True)

# In this notebook, we limit the number of files to be read in order to reduce the time required for model training.
defog_file_path = defog_file_path[::50]

print(f'the number of files to be read: {len(defog_file_path)}')

In [ ]:
# Initialize a DataFrame to combine data from multiple CSV files.
df_defog = pd.DataFrame()

for fp in tqdm.tqdm(defog_file_path):
    # load data into a variable 'tmp'.
    tmp = pd.read_csv(fp)
    
    # get file Id from csv file name.
    file_id = os.path.basename(fp).replace(".csv", "")
    
    # get subject Id.
    subject = df_defog_meta.loc[df_defog_meta['Id'] == file_id, 'Subject'].iloc[0]
    
    # add metadata.
    tmp['Medication'] = df_defog_meta.loc[df_defog_meta['Id'] == file_id, 'Medication'].iloc[0]
    tmp['Age'] = df_subjects.loc[df_subjects['Subject'] == subject, 'Age'].iloc[0]
    tmp['Sex'] = df_subjects.loc[df_subjects['Subject'] == subject, 'Sex'].iloc[0]
    tmp['YearsSinceDx'] = df_subjects.loc[df_subjects['Subject'] == subject, 'YearsSinceDx'].iloc[0]
    tmp['NFOGQ'] =df_subjects.loc[df_subjects['Subject'] == subject, 'NFOGQ'].iloc[0]
    
    # extract data from the time period where Valid and Task are both True.
    tmp = tmp[(tmp['Valid'] == True) & (tmp['Task']==True)]
    tmp = tmp.drop(['Valid', 'Task'], axis=1)
    
    # concat the data
    df_defog = pd.concat([df_defog, tmp]).reset_index(drop=True)

In [ ]:
# check the contents of the df_defog
df_defog.head()

# Prepare train data

In [ ]:
# concat tdcs and defog data.
df_train = pd.concat([df_tdcs, df_defog]).reset_index(drop=True)
df_train.head()

In [ ]:
# encode string columns into 0/1 format
df_train['Medication'] = np.where(df_train['Medication']=='on', 1, 0)
df_train['Sex'] = np.where(df_train['Sex']=='M', 1, 0)
df_train.head()

In [ ]:
# split data into features and target.
y = df_train[['StartHesitation', 'Turn', 'Walking']]                       # target
X = df_train.drop(['StartHesitation', 'Turn', 'Walking', 'Time'], axis=1)  # feature

In [ ]:
# check the contents of the target
y.head()

In [ ]:
# check the contents of the feature
X.head()

# Train RandomForestClassifier

In [ ]:
# train the model with default parameter
rf = RandomForestClassifier(random_state=0)
rf.fit(X, y)

# Prepare test data

## Process the test data in the same way as the training data

In [ ]:
# list of all tdcsfog csv file path
tdcs_test_file_path = glob.glob(os.path.join(pdir, 'test', 'tdcsfog', '*.csv'), recursive=True)
print(f'the number of files to be read: {len(tdcs_test_file_path)}')

In [ ]:
# Initialize a DataFrame to combine data from multiple CSV files.
df_tdcs_test = pd.DataFrame()

for fp in tqdm.tqdm(tdcs_test_file_path):
    
    # load data into a variable 'tmp'.
    tmp = pd.read_csv(fp)
    
    # get file Id from csv file name.
    file_id = os.path.basename(fp).replace(".csv", "")
    
    # get subject Id.
    subject = df_tdcs_meta.loc[df_tdcs_meta['Id'] == file_id, 'Subject'].iloc[0]
    
    # add metadata.
    tmp['Medication'] = df_tdcs_meta.loc[df_tdcs_meta['Id'] == file_id, 'Medication'].iloc[0]
    tmp['Age'] = df_subjects.loc[df_subjects['Subject'] == subject, 'Age'].iloc[0]
    tmp['Sex'] = df_subjects.loc[df_subjects['Subject'] == subject, 'Sex'].iloc[0]
    tmp['YearsSinceDx'] = df_subjects.loc[df_subjects['Subject'] == subject, 'YearsSinceDx'].iloc[0]
    tmp['NFOGQ'] =df_subjects.loc[df_subjects['Subject'] == subject, 'NFOGQ'].iloc[0]
    
    # add Id data to submit.
    tmp['Id'] = file_id + '_' + tmp['Time'].astype(str)
    
    # concat the data
    df_tdcs_test = pd.concat([df_tdcs_test, tmp]).reset_index(drop=True)

In [ ]:
# check the contents of the df_tdcs_test
df_tdcs_test.head()

In [ ]:
# list of all tdcsfog csv file path
defog_test_file_path = glob.glob(os.path.join(pdir, 'test', 'defog', '*.csv'), recursive=True)
print(f'the number of files to be read: {len(defog_test_file_path)}')

In [ ]:
# Initialize a DataFrame to combine data from multiple CSV files.
df_defog_test = pd.DataFrame()

for fp in tqdm.tqdm(defog_test_file_path):
    # load data into a variable 'tmp'.
    tmp = pd.read_csv(fp)
    
    # get file Id from csv file name.
    file_id = os.path.basename(fp).replace(".csv", "")
    
    # get subject Id.
    subject = df_defog_meta.loc[df_defog_meta['Id'] == file_id, 'Subject'].iloc[0]
    
    # add metadata.
    tmp['Medication'] = df_defog_meta.loc[df_defog_meta['Id'] == file_id, 'Medication'].iloc[0]
    tmp['Age'] = df_subjects.loc[df_subjects['Subject'] == subject, 'Age'].iloc[0]
    tmp['Sex'] = df_subjects.loc[df_subjects['Subject'] == subject, 'Sex'].iloc[0]
    tmp['YearsSinceDx'] = df_subjects.loc[df_subjects['Subject'] == subject, 'YearsSinceDx'].iloc[0]
    tmp['NFOGQ'] =df_subjects.loc[df_subjects['Subject'] == subject, 'NFOGQ'].iloc[0]
    
    # add Id data to submit.
    tmp['Id'] = file_id + '_' + tmp['Time'].astype(str)
    
    # concat the data
    df_defog_test = pd.concat([df_defog_test, tmp]).reset_index(drop=True)

In [ ]:
# check the contents of the df_defog_test
df_defog_test.head()

In [ ]:
# concat tdcs and defog data.
df_test = pd.concat([df_tdcs_test, df_defog_test]).reset_index(drop=True)

# encode string columns into 0/1 format
df_test['Medication'] = np.where(df_test['Medication']=='on', 1, 0)
df_test['Sex'] = np.where(df_test['Sex']=='M', 1, 0)
display(df_test)

In [ ]:
# split data into submission Id and feature.
Id = df_test['Id']                             # Id for submission data
X_test = df_test.drop(['Time', 'Id'], axis=1)  # feature of test data
X_test.head()

# Predict and submit

In [ ]:
# calculate prediction using trained RandomForestClassifier model.
prediction = rf.predict(X_test)

In [ ]:
# Prepare submit data
submit = pd.DataFrame(Id, columns=['Id'])
submit['StartHesitation'] = prediction[:, 0]
submit['Turn'] = prediction[:, 1]
submit['Walking'] = prediction[:, 2]

In [ ]:
display(submit)

In [ ]:
# Save the created submission data.
submit.to_csv('submission.csv', index=False)

<p style="font-size: 24px; font-weight: bold;">Congratulations!</p>

<p style="font-size: 16px;">You're now ready to submit your work on Kaggle!</p>

<p style="font-size: 16px;">Enjoy your experience on Kaggle!</p>